In [ ]:
import pandas as pd
import numpy as np
df=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
df['subscribe'].value_counts() 


: 

In [ ]:
import pandas as pd

file_path = 'train.csv'

file_path1 = 'test.csv'
# 将 CSV 文件加载到 pandas DataFrame 中
df = pd.read_csv(file_path)

# 检查缺失值
missing_values = df.isnull().sum()

# 打印每列的缺失值数量'''  '''
print("每列的缺失值数量:")
print(missing_values)


: 

In [ ]:

import pandas as pd
import numpy as np
df=pd.read_csv("train.csv")

import matplotlib.pyplot as plt       
import seaborn as sns
bins=[0,143,353,1873,5149]
df1=df[df['subscribe']=='yes']
binning=pd.cut(df1['duration'],bins,right=False)
time=pd.value_counts(binning)
# 可视化
time=time.sort_index()
fig=plt.figure(figsize=(6,2),dpi=120)
sns.barplot(x=time.index, y=time.values, color='royalblue')

x=np.arange(len(time))
y=time.values
for x_loc,jobs in zip(x,y):
    plt.text(x_loc, jobs+2, '{:.1f}%'.format(jobs/sum(time)*100), ha='center', va= 'bottom',fontsize=8)
plt.xticks(fontsize=8)
plt.yticks([])
plt.ylabel('')
plt.title('duration_yes',size=8)
sns.despine(left=True)
plt.show()

: 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 读取数据
df = pd.read_csv("train.csv")

# 定义区间
bins = [0, 143, 353, 1873, 5149]

# 根据 'duration' 列进行分组
df['duration_bin'] = pd.cut(df['duration'], bins, right=False)

# 筛选 'subscribe' 列为 'yes' 的数据
df_yes = df[df['subscribe'] == 'yes']

# 使用 groupby 计算每个区间内 'yes' 的数量
yes_counts = df_yes.groupby('duration_bin').size().reset_index(name='yes_count')

# 输出结果
print(yes_counts)


: 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 读取数据
df = pd.read_csv("train.csv")SA

# 分离数值变量与分类变量
Nu_feature = list(df.select_dtypes(exclude=['object']).columns)  
Ca_feature = list(df.select_dtypes(include=['object']).columns)
#查看训练集与测试集数值变量分布
import matplotlib.pyplot as plt       
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
plt.figure(figsize=(20,15))
i=1
for col in Nu_feature:
    ax=plt.subplot(4,4,i)
    ax=sns.kdeplot(df[col],color='red')
    ax=sns.kdeplot(test[col],color='cyan')
    ax.set_xlabel(col)
    ax.set_ylabel('Frequency')
    ax=ax.legend(['train','test'])
    i+=1
plt.show()
# 查看分类变量分布
Ca_feature.remove('subscribe')
col1=Ca_feature
plt.figure(figsize=(20,10))
j=1
for col in col1:
    ax=plt.subplot(4,5,j)
    ax=plt.scatter(x=range(len(df)),y=df[col],color='red')
    plt.title(col)
    j+=1
k=11
for col in col1:
    ax=plt.subplot(4,5,k)
    ax=plt.scatter(x=range(len(test)),y=test[col],color='cyan')
    plt.title(col)
    k+=1
plt.subplots_adjust(wspace=0.4,hspace=0.3)  
plt.show()

: 

In [ ]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 读取数据
df = pd.read_csv("train.csv")
lb = LabelEncoder()

# 假设 Ca_feature 是你的某些列的列表
cols = Ca_feature
for m in cols:
    df[m] = lb.fit_transform(df[m])
    # 假设 test 是你的测试数据集
    test[m] = lb.fit_transform(test[m])

df['subscribe'] = df['subscribe'].replace(['no', 'yes'], [0, 1])

correlation_matrix = df.corr()

# 设置样式为"whitegrid"
sns.set_style("whitegrid")

# 设置图形大小
plt.figure(figsize=(12, 10))

# 绘制热力图
sns.heatmap(correlation_matrix, vmax=0.9, linewidths=0.05, cmap="coolwarm", annot=True, fmt=".2f")


# 显示图形
plt.show()


: 

In [ ]:
from lightgbm.sklearn import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, auc, roc_auc_score
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 读取数据
df = pd.read_csv("train.csv")
lb = LabelEncoder()
X=df.drop(columns=['id','subscribe'])
Y=df['subscribe']
test=test.drop(columns='id')
# 划分训练及测试集
x_train,x_test,y_train,y_test = train_test_split( X, Y,test_size=0.3,random_state=1)
# 建立模型
gbm = LGBMClassifier(n_estimators=600,learning_rate=0.01,boosting_type= 'gbdt',   
    objective = 'binary',
    max_depth = -1,  
    random_state=2022,           
    metric='auc')
# 交叉验证
result1 = []
mean_score1 = 0
n_folds=5
kf = KFold(n_splits=n_folds ,shuffle=True,random_state=2022)
for train_index, test_index in kf.split(X):
    x_train = X.iloc[train_index]
    y_train = Y.iloc[train_index]
    x_test = X.iloc[test_index]
    y_test = Y.iloc[test_index]
    gbm.fit(x_train,y_train)
    y_pred1=gbm.predict_proba((x_test),num_iteration=gbm.best_iteration_)[:,1]
    print('验证集AUC:{}'.format(roc_auc_score(y_test,y_pred1)))
    mean_score1 += roc_auc_score(y_test,y_pred1)/ n_folds
    y_pred_final1 = gbm.predict_proba((test),num_iteration=gbm.best_iteration_)[:,1]
    y_pred_test1=y_pred_final1
    result1.append(y_pred_test1)
# 模型评估
print('mean 验证集auc:{}'.format(mean_score1))
cat_pre1=sum(result1)/n_folds
ret1=pd.DataFrame(cat_pre1,columns=['subscribe'])
ret1['subscribe']=np.where(ret1['subscribe']>0.5,'yes','no').astype('str')
ret1.to_csv('GBM预测.csv',index=False)

: 